<h1>Импорты</h1>

In [1547]:
import pandas as pd
import numpy as np
import pyodbc
import random
from sklearn.preprocessing import OrdinalEncoder
from datetime import datetime

In [1548]:
db_conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=LAPTOP-JSR6TV0G;'
                      'Database=Pro_t_londonSQL;'
                      'Trusted_Connection=yes;')

<h1>Postgres</h1>

In [1549]:
from psycopg2 import connect, sql, DatabaseError
import psycopg2.extras as extras

In [1550]:
from psycopg2.extensions import register_adapter, AsIs
register_adapter(np.int32, AsIs)
register_adapter(np.int64, AsIs)

In [1551]:
# Connect
try:
    conn = connect (
        dbname = "documentskm",
        user = "postgres",
        host = "localhost",
        password = "password"
    )
    cursor = conn.cursor()
except Exception as err:
    cursor = None
    print("Psycopg2 error:", err)
    
# Check if the connection was valid
if cursor != None:
    print("Connection successful")

Connection successful


In [1552]:
def execute_values(conn, df, table):
    tuples = [tuple(x) for x in df.to_numpy()]
    cols = ','.join(list(df.columns))
    query  = "INSERT INTO %s(%s) VALUES %%s" % (table, cols)
    cursor = conn.cursor()
    try:
        extras.execute_values(cursor, query, tuples)
        conn.commit()
    except (Exception, DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    print("execute_values() done")
    cursor.close()

In [1553]:
def set_sequence(conn, table):
    cursor = conn.cursor()
    try:
        cursor.execute("select setval('{}_id_seq', (select max(id) from {}));".format(table, table))
        v = cursor.fetchone()
        print(v)
        cursor.close()
    except (Exception, DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1

<h1>Id sequence will be out of sync</h1>

select setval('mark_linked_docs_id_seq', (select max(id) from mark_linked_docs));

select setval('general_data_points_id_seq', (select max(id) from general_data_points));

select setval('mark_general_data_points_id_seq', (select max(id) from mark_general_data_points));

select setval('general_data_points_id_seq', (select max(id) from general_data_points));

<h2>If you need to drop all tables</h2>

drop schema public cascade;
create schema public;

<h1>Должности</h1>

In [1554]:
df = pd.read_sql_query('SELECT * FROM Pro_t_londonSQL.dbo.Спр_должностей', db_conn)

In [1555]:
df.at[13, "наим_должн"] = "инж. 3 к"

In [1556]:
df = df.rename(columns={"должность": "id",
                        "наим_должн": "name"
                       })

In [1557]:
cursor.execute(open("sql/25.sql", "r").read())
conn.commit()

In [1558]:
execute_values(conn, df, "positions")

execute_values() done


In [1559]:
set_sequence(conn, "positions")

(17,)


<h1>Отделы</h1>

In [1560]:
df = pd.read_sql_query('SELECT * FROM Pro_t_londonSQL.dbo.Спр_отделов', db_conn)

In [1561]:
df = df.drop(["начальник", "шифр_отд"], axis=1)

In [1562]:
df["наименование"] = df["наименование"].str.strip()

In [1563]:
df = df[df["код_отд"] != 40]
df = df[df["код_отд"] != 37]
df = df[df["код_отд"] != 36]
df = df[df["код_отд"] != 35]
df = df[df["код_отд"] != 34]
df = df[df["код_отд"] != 33]
df = df[df["код_отд"] != 32]
df = df[df["код_отд"] != 31]
df = df[df["код_отд"] != 18]
df = df[df["код_отд"] != 27]

In [1564]:
df.at[0, "наименование"] = "СО-2б"

In [1565]:
df.at[1, "наименование"] = "Машиностроительный"

In [1566]:
df = df[df["код_отд"] != 29]

In [1567]:
df.at[30, "наименование"] = "СО-2а"

In [1568]:
df = df.rename(columns={"код_отд": "id",
                        "наименование": "name"
                       })

In [1569]:
cursor.execute(open("sql/30.sql", "r").read())
conn.commit()

In [1570]:
execute_values(conn, df, "departments")

execute_values() done


In [1571]:
set_sequence(conn, "departments")

(39,)


<h1>Исполнители</h1>

In [1572]:
df = pd.read_sql_query('SELECT * FROM Pro_t_londonSQL.dbo.Спр_исполн', db_conn)

In [1573]:
df = df.dropna()

In [1574]:
df.loc[:,"код_отд"] = df.loc[:,"код_отд"].replace(40, 1)
df.loc[:,"код_отд"] = df.loc[:,"код_отд"].replace(37, 1)
df.loc[:,"код_отд"] = df.loc[:,"код_отд"].replace(36, 1)
df.loc[:,"код_отд"] = df.loc[:,"код_отд"].replace(35, 26)
df.loc[:,"код_отд"] = df.loc[:,"код_отд"].replace(34, 1)
df.loc[:,"код_отд"] = df.loc[:,"код_отд"].replace(33, 26)
df.loc[:,"код_отд"] = df.loc[:,"код_отд"].replace(32, 1)
df.loc[:,"код_отд"] = df.loc[:,"код_отд"].replace(31, 26)
df.loc[:,"код_отд"] = df.loc[:,"код_отд"].replace(18, 9)
df.loc[:,"код_отд"] = df.loc[:,"код_отд"].replace(27, 26)
df.loc[:,"код_отд"] = df.loc[:,"код_отд"].replace(29, 3)

In [1575]:
df = df[df["код_отд"] != 0]

In [1576]:
df["должность"] = df["должность"].str.strip()

In [1577]:
df.loc[df["должность"] == "ген.дир", 'тип_должн'] = 1
df.loc[df["должность"] == "гл.инж", 'тип_должн'] = 2
df.loc[df["должность"] == "зам.гл.инж", 'тип_должн'] = 3
df.loc[df["должность"] == "гл.стр", 'тип_должн'] = 4
df.loc[df["должность"] == "гл.арх", 'тип_должн'] = 5
df.loc[df["должность"] == "ГИП", 'тип_должн'] = 6
df.loc[df["должность"] == "нач.отд", 'тип_должн'] = 7
df.loc[df["должность"] == "зам.нач.отд", 'тип_должн'] = 8
df.loc[df["должность"] == "гл.спец.", 'тип_должн'] = 9
df.loc[df["должность"] == "Гл. спец.", 'тип_должн'] = 9
df.loc[df["должность"] == "зав.гр", 'тип_должн'] = 10
df.loc[df["должность"] == "Зав.гр", 'тип_должн'] = 10
df.loc[df["должность"] == "зав. Гр", 'тип_должн'] = 10
df.loc[df["должность"] == "вед.инж", 'тип_должн'] = 11
df.loc[df["должность"] == "вед.инж.", 'тип_должн'] = 11
df.loc[df["должность"] == "инж. 1 к", 'тип_должн'] = 12
df.loc[df["должность"] == "Инж. 1к", 'тип_должн'] = 12
df.loc[df["должность"] == "инженер1к", 'тип_должн'] = 12
df.loc[df["должность"] == "инж. 2 к", 'тип_должн'] = 13
df.loc[df["должность"] == "инж. 3 к", 'тип_должн'] = 14
df.loc[df["должность"] == "Инж. 3 к", 'тип_должн'] = 14
df.loc[df["должность"] == "инж3кат", 'тип_должн'] = 14
df.loc[df["должность"] == "инженер", 'тип_должн'] = 15
df.loc[df["должность"] == "инж", 'тип_должн'] = 15
df.loc[df["должность"] == "техник", 'тип_должн'] = 16
df.loc[df["должность"] == "чертежн", 'тип_должн'] = 17

In [1578]:
df["фио"] = df["фио"].str.strip()

In [1579]:
df = df.drop_duplicates(subset=["тип_должн", "фио", "код_отд"])

In [1580]:
df = df[df["тип_должн"] != 0]

In [1581]:
df = df.drop(["шифр", "должность"], axis=1)

In [1582]:
df = df.rename(columns={"код": "id",
                        "тип_должн": "position_id",
                        "фио": "name",
                        "код_отд": "department_id"
                       })

In [1583]:
df["id"].to_csv("employee_ids.csv")

In [1584]:
cursor.execute(open("sql/26.sql", "r").read())
conn.commit()

In [1585]:
execute_values(conn, df, "employees")

execute_values() done


In [1586]:
set_sequence(conn, "employees")

(646,)


<h1>Пользователи</h1>

In [1587]:
cursor.execute(open("sql/xx_users.sql", "r").read())
conn.commit()

<h1>Условия эксплуатации</h1>

In [1588]:
df1 = pd.DataFrame({'id': [1, 2, 3, 4], 'name': ["неагрессивная", "слабоагрессивная", "среднеагрессивная", "сильноагрессивная"]})

In [1589]:
df2 = pd.DataFrame({'id': [1, 2, 3], 'name': ["внутри помещения", "снаружи помещения", "в жидкостях"]})

In [1590]:
df3 = pd.DataFrame({'id': [1, 2, 3, 4, 5], 'name': ["нет газов (вода)", "газы группы А (кислоты)", "газы группы B, C и D (щелочи)", "бензин", "масло"]})

In [1591]:
df4 = pd.DataFrame({'id': [1, 2, 3, 4], 'name': ["сталь без покрытия", "оцинкованная сталь", "сталь горячего цинкования", "сталь с газотермическим напылением цинка"]})

In [1592]:
df5 = pd.DataFrame({'id': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16], 'name': ["ПФ", "АУ", "ЭЭ", "МА", "МБ", "НЦ", "ФФ", "ПА", "ПЭ", "ПБ", "ПВ", "СВ", "ПУ", "ЭП", "ПС", "КО"]})

In [1593]:
df6 = pd.DataFrame({'id': [1, 2, 3, 4], 'name': ["отсутствуют", "сдвигоустойчивые соединения", "фланцевые соединения", "сдвигоустойчивые и фланцевые соединения"]})

In [1594]:
cursor.execute(open("sql/y_1.sql", "r").read())
cursor.execute(open("sql/y_2.sql", "r").read())
cursor.execute(open("sql/y_3.sql", "r").read())
cursor.execute(open("sql/y_4.sql", "r").read())
cursor.execute(open("sql/y_5.sql", "r").read())
cursor.execute(open("sql/y_6.sql", "r").read())
conn.commit()

In [1595]:
execute_values(conn, df1, "env_aggressiveness")
execute_values(conn, df2, "operating_areas")
execute_values(conn, df3, "gas_groups")
execute_values(conn, df4, "construction_materials")
execute_values(conn, df5, "paintwork_types")
execute_values(conn, df6, "high_tensile_bolts_types")

execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done
execute_values() done


In [1596]:
set_sequence(conn, "env_aggressiveness")
set_sequence(conn, "operating_areas")
set_sequence(conn, "gas_groups")
set_sequence(conn, "construction_materials")
set_sequence(conn, "paintwork_types")
set_sequence(conn, "high_tensile_bolts_types")

(4,)
(3,)
(5,)
(4,)
(16,)
(4,)


<h1>Контроль сварки</h1>

In [1597]:
df = pd.DataFrame({'id': [1, 2, 3, 4], 'name': ["не требуется", "опрыскиванием керосином", "ультразвуком", "гамма-излучением"]})

In [1598]:
cursor.execute(open("sql/yy_1.sql", "r").read())
conn.commit()

In [1599]:
execute_values(conn, df, "welding_control")

execute_values() done


In [1600]:
set_sequence(conn, "welding_control")

(4,)


<h1>Типы документов</h1>

In [1601]:
df = pd.DataFrame({'id': [1, 2, 3, 4, 5], 'code': ["л.", "СМ", "ВМП", "СМС", "РР"], 'name': ["Листы основного комплекта", "Спецификация металлопроката", "Ведомость металлоконструкций по видам профилей", "Сводная спецификация металлопроката", "Расчет металлоконструкций"]})

In [1602]:
cursor.execute(open("sql/x.sql", "r").read())
conn.commit()

In [1603]:
execute_values(conn, df, "doc_types")

execute_values() done


In [1604]:
set_sequence(conn, "doc_types")

(5,)


<h1>Степени очистки антикоррозионной защиты</h1>

In [1605]:
df = pd.read_sql_query('SELECT * FROM Pro_t_londonSQL.dbo.Степени_оч_аз', db_conn)

In [1606]:
df["Наим_оч"] = df["Наим_оч"].str.strip()

In [1607]:
df = df.rename(columns={"Степень_оч": "id",
                        "Наим_оч": "name"
                       })

In [1608]:
cursor.execute(open("sql/44.sql", "r").read())
conn.commit()

In [1609]:
execute_values(conn, df, "corr_prot_cleaning_degrees")

execute_values() done


In [1610]:
set_sequence(conn, "corr_prot_cleaning_degrees")

(4,)


<h1>Варианты антикоррозионной защиты </h1>

In [1611]:
df = pd.read_sql_query('SELECT * FROM Pro_t_londonSQL.dbo.Варианты_аз', db_conn)

In [1612]:
df = df.replace("", np.nan)

In [1613]:
df = df.dropna()

In [1614]:
df = df.drop_duplicates()

In [1615]:
df = df.drop(["Статус_аз"], axis=1)

In [1616]:
df.insert(0, 'id', range(1, len(df) + 1))

In [1617]:
df = df.drop_duplicates(subset=["Зона_экс", "Группа_газов", "Агресс_ср", "Матер_к", "Тип_лп", "Группа_лп", "Стойкость_лп"])

In [1618]:
df = df.rename(columns={"Зона_экс": "operating_zone",
                        "Группа_газов": "gas_group",
                        "Агресс_ср": "env_aggressiveness",
                        "Матер_к": "material",
                        "Тип_лп": "paintwork_type",
                        "Группа_лп": "paintwork_group",
                        "Стойкость_лп": "paintwork_durability",
                        "Кол_слоев_лп": "paintwork_num_of_layers",
                        "Толщ_лпгр": "paintwork_primer_thickness",
                        "Кол_слоев_гр": "primer_num_of_layers",
                        "Степень_оч": "cleaning_degree_id"
                       })

In [1619]:
cursor.execute(open("sql/1.sql", "r").read())
conn.commit()

In [1620]:
execute_values(conn, df, "corr_prot_variants")

execute_values() done


In [1621]:
set_sequence(conn, "corr_prot_variants")

(204,)


<h1>Виды конструкций</h1>

In [1622]:
df = pd.read_sql_query('SELECT * FROM Pro_t_londonSQL.dbo.Спр_видов_к', db_conn)

In [1623]:
df["Имя_к"] = df["Имя_к"].str.strip()

In [1624]:
df = df[df["Имя_к"].isna() == False]

In [1625]:
df = df.rename(columns={"Шифр_к": "id",
                        "Имя_к": "name"
                       })

In [1626]:
df.loc[df["id"] == 2, "name"] = "Двухветвевые колонны одноэтажных производственных зданий и крановых эстакад"
df.loc[df["id"] == 9, "name"] = "Подкраново-подстропильные фермы"
df.loc[df["id"] == 34, "name"] = "Переплеты, витражи из профильных труб с деталями крепления"

In [1627]:
cursor.execute(open("sql/22.sql", "r").read())
conn.commit()

In [1628]:
execute_values(conn, df, "construction_types")

execute_values() done


In [1629]:
set_sequence(conn, "construction_types")

(87,)


<h1>Подвиды конструкций</h1>

In [1630]:
df = pd.read_sql_query('SELECT * FROM Pro_t_londonSQL.dbo.Спр_видов_пк', db_conn)

In [1631]:
df["Имя_пк"] = df["Имя_пк"].str.strip()
df["Расценка"] = df["Расценка"].str.strip()

In [1632]:
df = df.drop_duplicates()

In [1633]:
df = df.drop(["Шифр_пк"], axis=1)

In [1634]:
df.insert(0, 'id', range(1, len(df) + 1))

In [1635]:
df = df.rename(columns={"Шифр_к": "type_id",
                        "Имя_пк": "name",
                        "Расценка": "valuation"
                       })

In [1636]:
df.loc[df["id"] == 11, "name"] = "высотой свыше 150 м, поставляемые отдельными элементами"
df.loc[df["id"] == 14, "name"] = "коробчатого сечения пролетом 12 м и более"
df.loc[df["id"] == 33, "name"] = "с шагом стропильных ферм 6 м"
df.loc[df["id"] == 34, "name"] = "с шагом стропильных ферм 12 м"
df.loc[df["id"] == 49, "name"] = "составного сечения из горячекатаных профилей, соединенных планками"
df.loc[df["id"] == 51, "name"] = "ветви из двутавров с парал. гранями полок, решетка из г/кат. уголков и швеллеров"
df.loc[df["id"] == 52, "name"] = "ветви из двутавров (ГОСТ 8239-72) или швеллеров и решетка из г/кат. уголков"
df.loc[df["id"] == 69, "name"] = "с поясами из тавров, стенкой из листовой стали, пролетом 6 м, массой 1 м,т:"
df.loc[df["id"] == 70, "name"] = "с поясами из тавров, стенкой из листовой стали, пролетом 12 м и более,т:"
df.loc[df["id"] == 71, "name"] = "из листовой стали пролетом 6 м, массой 1 м,т:"
df.loc[df["id"] == 72, "name"] = "из листовой стали пролетом 12 м, массой 1 м,т:"
df.loc[df["id"] == 83, "name"] = "пролетом 12 м, решетчатые из г/кат. и гнутых профилей, масса 1 м до 0.4 т"
df.loc[df["id"] == 84, "name"] = "пролетом 12 м, решетчатые из г/кат. и гнутых профилей, масса 1 м свыше 0.4 т"
df.loc[df["id"] == 85, "name"] = "пролетом 12 м, решетчатые сост. сечения из листовой стали, масса 1 м до 0.4 т"
df.loc[df["id"] == 86, "name"] = "пролетом 12 м, решетчатые сост. сечения из листовой стали, масса 1 м свыше 0.4 т"
df.loc[df["id"] == 87, "name"] = "составного сечения из листовой стали, масса 1 м,т:"
df.loc[df["id"] == 88, "name"] = "составного сечения из двух и более уголков или швеллеров, масса 1 м,т:"
df.loc[df["id"] == 89, "name"] = "из двутавров с параллельными гранями полок, масса 1 м,т:"
df.loc[df["id"] == 90, "name"] = "составного сечения из листовой стали с верхним опиранием настила, масса марки,т:"
df.loc[df["id"] == 91, "name"] = "составного сечения при пониженном опирании настила, масса марки,т:"
df.loc[df["id"] == 92, "name"] = "из двутавров с парал. гранями полок с верхним опиранием настила, масса до 1 т"
df.loc[df["id"] == 93, "name"] = "из двутавров с парал. гранями полок с верхним опиранием настила, масса свыше 1 т"
df.loc[df["id"] == 94, "name"] = "из двутавров с парал. гранями полок с пониж. опиранием настила, масса до 1 т"
df.loc[df["id"] == 95, "name"] = "из двутавров с парал. гранями полок с пониж. опиранием настила, масса свыше 1 т"
df.loc[df["id"] == 113, "name"] = "фермы из парных уголков и тавров, попер. балки из двутавров, масса 1 м до 0.5 т"
df.loc[df["id"] == 114, "name"] = "фермы из парных уголков и тавров, попер. балки из двутавров, масса 1 м до 1 т"
df.loc[df["id"] == 115, "name"] = "фермы из парных уголков и тавров, попер. балки из двутавров, масса 1 м свыше 1 т"

In [1637]:
cursor.execute(open("sql/24.sql", "r").read())
conn.commit()

In [1638]:
execute_values(conn, df, "construction_subtypes")

execute_values() done


In [1639]:
set_sequence(conn, "construction_subtypes")

(125,)


<h1>Типовые наименования листов</h1>

In [1640]:
df = pd.read_sql_query('SELECT * FROM Pro_t_londonSQL.dbo.Спр_листов_н', db_conn)

In [1641]:
df["ш_листа"] = df["ш_листа"].str.strip()

In [1642]:
df = df.rename(columns={"ш_номер": "id",
                        "ш_листа": "name"
                       })

In [1643]:
cursor.execute(open("sql/27.sql", "r").read())
conn.commit()

In [1644]:
execute_values(conn, df, "sheet_names")

execute_values() done


In [1645]:
set_sequence(conn, "sheet_names")

(385,)


<h1>Виды ссылочных документов</h1>

In [1646]:
df = pd.read_sql_query('SELECT * FROM Pro_t_londonSQL.dbo.Спр_ссыл_в', db_conn)

In [1647]:
df["Наим_вида"] = df["Наим_вида"].str.strip()
df["Наим_вида"] = df["Наим_вида"].str.capitalize()

In [1648]:
df = df.drop_duplicates(subset=["Наим_вида"])

In [1649]:
df = df.rename(columns={"Вид_сд": "id",
                        "Наим_вида": "name"
                       })

In [1650]:
cursor.execute(open("sql/40.sql", "r").read())
conn.commit()

In [1651]:
execute_values(conn, df, "linked_doc_types")

execute_values() done


In [1652]:
set_sequence(conn, "linked_doc_types")

(29,)


<h1>Ссылочные документы</h1>

In [1653]:
df = pd.read_sql_query('SELECT * FROM Pro_t_londonSQL.dbo.Спр_ссыл', db_conn)

In [1654]:
df = df.replace("", np.nan)

In [1655]:
df = df.dropna()

In [1656]:
df = df.drop_duplicates(subset=["Шифр_сд"])

In [1657]:
df = df.reset_index(drop=True)

In [1658]:
df["Шифр_сд"] = df["Шифр_сд"].str.strip()
df["Обозн_сд"] = df["Обозн_сд"].str.strip()
df["Наим_сд"] = df["Наим_сд"].str.strip()

In [1659]:
for i in range(0, len(df)):
    arr = df["Наим_сд"][i].split()
    new_str = ' '.join(arr)
    df.at[i, "Наим_сд"] = new_str

In [1660]:
df["Наим_сд"] = df["Наим_сд"].str.replace("возду ха", "воздуха")
df["Наим_сд"] = df["Наим_сд"].str.replace("Материриалы", "Материалы")
df["Наим_сд"] = df["Наим_сд"].str.replace("рекоментации", "рекомендации")
df["Наим_сд"] = df["Наим_сд"].str.replace("рас пашные", "распашные")
df["Наим_сд"] = df["Наим_сд"].str.replace("металличе ческих", "металлических")
df["Наим_сд"] = df["Наим_сд"].str.replace("металличе ских", "металлических")
df["Наим_сд"] = df["Наим_сд"].str.replace("электри ческие", "электрические")
df["Наим_сд"] = df["Наим_сд"].str.replace("кра ны", "краны")
df["Наим_сд"] = df["Наим_сд"].str.replace("примене нием", "применением")
df["Наим_сд"] = df["Наим_сд"].str.replace("обслуживани сосудов", "обслуживания сосудов")
df["Наим_сд"] = df["Наим_сд"].str.replace("пе реплетами", "переплетами")
df["Наим_сд"] = df["Наим_сд"].str.replace("за полнением", "заполнением")
df["Наим_сд"] = df["Наим_сд"].str.replace("линй", "линий")
df["Наим_сд"] = df["Наим_сд"].str.replace("мосто вые", "мостовые")
df["Наим_сд"] = df["Наим_сд"].str.replace("покрыти ем", "покрытием")
df["Наим_сд"] = df["Наим_сд"].str.replace("со кращенному", "сокращенному")
df["Наим_сд"] = df["Наим_сд"].str.replace("покрытияпролетами", "покрытия пролетами")
df["Наим_сд"] = df["Наим_сд"].str.replace("широкополочныых", "широкополочных")
df["Наим_сд"] = df["Наим_сд"].str.replace("тав ров", "тавров")
df["Наим_сд"] = df["Наим_сд"].str.replace("зда ний", "зданий")
df["Наим_сд"] = df["Наим_сд"].str.replace("ремонтно -монтажных", "ремонтно-монтажных")
df["Наим_сд"] = df["Наим_сд"].str.replace("пок рытия", "покрытия")
df["Наим_сд"] = df["Наим_сд"].str.replace("профилирован ным", "профилированным")
df["Наим_сд"] = df["Наим_сд"].str.replace("стеклои минераловатных", "стекло и минераловатных")
df["Наим_сд"] = df["Наим_сд"].str.replace("окон,дверей", "окон, дверей")
df["Наим_сд"] = df["Наим_сд"].str.replace("пром.зданий", "пром. зданий")
df["Наим_сд"] = df["Наим_сд"].str.replace("строитель ные", "строительные")
df["Наим_сд"] = df["Наим_сд"].str.replace("сталь ные", "стальные")
df["Наим_сд"] = df["Наим_сд"].str.replace("наруж ного", "наружного")
df["Наим_сд"] = df["Наим_сд"].str.replace("ветров.район.", "ветров. район.")
df["Наим_сд"] = df["Наим_сд"].str.replace("выше и ниже -40 град.с.  ше и ниже -40 град.с.", "выше -40 град.с. и ниже -40 град.с.")
df["Наим_сд"] = df["Наим_сд"].str.replace("алю миниевых", "алюминиевых")
df["Наим_сд"] = df["Наим_сд"].str.replace("зданий,промышленных", "зданий, промышленных")
df["Наим_сд"] = df["Наим_сд"].str.replace("предприятий.шарнирные", "предприятий. Шарнирные")
df["Наим_сд"] = df["Наим_сд"].str.replace("связей.Чертежи", "связей. Чертежи")
df["Наим_сд"] = df["Наим_сд"].str.replace("сталь ными", "стальными")
df["Наим_сд"] = df["Наим_сд"].str.replace("унифици рованных", "унифицированных")
df["Наим_сд"] = df["Наим_сд"].str.replace("уз лы", "узлы")
df["Наим_сд"] = df["Наим_сд"].str.replace("железобе тонных", "железобетонных")
df["Наим_сд"] = df["Наим_сд"].str.replace("метал лопроката", "металлопроката")
df["Наим_сд"] = df["Наим_сд"].str.replace("выпу ску", "выпуску")
df["Наим_сд"] = df["Наим_сд"].str.replace("откры вающиеся", "открывающиеся")
df["Наим_сд"] = df["Наим_сд"].str.replace("светоаэра ционных", "светоаэрационных")
df["Наим_сд"] = df["Наим_сд"].str.replace("проектирования.Рабочие", "проектирования. Рабочие")
df["Наим_сд"] = df["Наим_сд"].str.replace("черте жи", "чертежи")
df["Наим_сд"] = df["Наим_сд"].str.replace("опорн.кранами", "опорн. кранами")
df["Наим_сд"] = df["Наим_сд"].str.replace("опорынми кранами", "опорными кранами")
df["Наим_сд"] = df["Наим_сд"].str.replace("несущ.конструкций", "несущ. конструкций")
df["Наим_сд"] = df["Наим_сд"].str.replace("подв.кранами", "подв. кранами")
df["Наим_сд"] = df["Наим_сд"].str.replace("кровлей,оборудованных", "кровлей, оборудованных")
df["Наим_сд"] = df["Наим_сд"].str.replace("одноэт.произв", "одноэт. произв")
df["Наим_сд"] = df["Наим_сд"].str.replace("произв.зданий", "произв. зданий")
df["Наим_сд"] = df["Наим_сд"].str.replace("кровлей,оборудованных", "кровлей, оборудованных")
df["Наим_сд"] = df["Наим_сд"].str.replace("специальн.назначения", "специальн. назначения")
df["Наим_сд"] = df["Наим_сд"].str.replace("зданий,оборудованных", "зданий, оборудованных")
df["Наим_сд"] = df["Наим_сд"].str.replace("две рей", "дверей")

In [1661]:
df = df.drop_duplicates(subset=["Наим_сд"])

In [1662]:
df.loc[df["Наим_сд"].str.endswith('.') == False, "Наим_сд"] = df[df["Наим_сд"].str.endswith('.') == False]["Наим_сд"] + '.'

In [1663]:
df.insert(0, 'id', range(1, len(df) + 1))

In [1664]:
df = df.rename(columns={"Шифр_сд": "code",
                        "Вид_сд": "type_id",
                        "Обозн_сд": "designation",
                        "Наим_сд": "name"
                       })

In [1665]:
cursor.execute(open("sql/39.sql", "r").read())
conn.commit()

In [1666]:
execute_values(conn, df, "linked_docs")

execute_values() done


In [1667]:
set_sequence(conn, "linked_docs")

(305,)


<h1>Категории производства по пожарной и взрывопожарной опасности</h1>

In [1668]:
df = pd.read_sql_query('SELECT * FROM Pro_t_londonSQL.dbo.Категории_пр', db_conn)

In [1669]:
df = df.replace("", np.nan)

In [1670]:
df.insert(0, 'id', range(1, len(df) + 1))

In [1671]:
df = df.rename(columns={"Категория": "category",
                        "Название": "name",
                        "Текст_од": "description"
                       })

In [1672]:
df = df.where(pd.notnull(df), None)

In [1673]:
cursor.execute(open("sql/5.sql", "r").read())
conn.commit()

In [1674]:
execute_values(conn, df, "fire_hazard_categories")

execute_values() done


In [1675]:
set_sequence(conn, "fire_hazard_categories")

(6,)


<h1>Проекты</h1>

In [1676]:
df = pd.read_sql_query('SELECT * FROM Pro_t_2.dbo.Марки', db_conn)
df.shape

(4112, 33)

In [1677]:
df = df.dropna(subset=["код_марки", "обозн", "комплекс", "объект"])

In [1678]:
e_ids = pd.read_csv("employee_ids.csv")
e_ids = e_ids.drop(["Unnamed: 0"], axis=1)
e_ids = e_ids.values.flatten().tolist()
df.loc[df["гл_спец"].isin(e_ids) == False, "гл_спец"] = None
df.loc[df["рук_гр"].isin(e_ids) == False, "рук_гр"] = None
df.loc[df["н_контр"].isin(e_ids) == False, "н_контр"] = None

In [1679]:
df = df[df["обозн"].str.contains("демонтаж") == False]
df = df[df["обозн"].str.contains("Мновый") == False]
df["обозн"] = df["обозн"].str.replace(",", ".")

In [1680]:
df2 = pd.DataFrame(df["обозн"].str.split('-КМ', 1).tolist(),
                                 columns = ['first','code'])

In [1681]:
df = df.reset_index()
df2 = df2.join(df)

In [1682]:
df2 = df2.dropna(subset=["code"])

In [1683]:
df2["code"] = "КМ" + df2["code"]

In [1684]:
df3 = pd.DataFrame(df2["first"].str.split('.', 1).tolist(),
                                 columns = ['baseSeries','middle'])

In [1685]:
df2 = df2.reset_index()
df3 = df3.reset_index()
df4 = df2.join(df3[["baseSeries", "middle"]])

In [1686]:
df4 = df4.drop(["first", "index"], axis=1)
df4 = df4.rename(columns={"код_марки": "id"})

In [1687]:
df5 = pd.DataFrame(df4["middle"].fillna("").str.split('.', 1).tolist(),
                                 columns = ['node','subnode'])

In [1688]:
df6 = pd.DataFrame(df5["subnode"].fillna("").str.split('-', 1).tolist(),
                                 columns = ['subnode_part','node_part'])

In [1689]:
df5 = df5.reset_index()
df6 = df6.reset_index()
df7 = df5.join(df6[["subnode_part", "node_part"]])

In [1690]:
df7["node_part"] = "-" + df7["node_part"]
df7["node2"] = df7["node"] + df7["node_part"].fillna("")

In [1691]:
df4 = df4.reset_index(drop=True)
df7 = df7.reset_index(drop=True)
df8 = df4.join(df7[["node2", "subnode_part"]])

In [1692]:
df8 = df8.rename(columns={"node2": "nodeCode", "subnode_part": "subnodeCode"})
df8 = df8.drop(["middle"], axis=1)

In [1693]:
enc = OrdinalEncoder()
res = enc.fit_transform(np.array(df8["baseSeries"]).reshape(-1, 1))

In [1694]:
s = pd.DataFrame(pd.Series(res.flatten()) + 1)
s = s.rename(columns={0: "project_id"})

In [1695]:
df9 = df8.join(s)

In [1696]:
projects_df = df9[["baseSeries", "комплекс", "project_id"]]

In [1697]:
projects_df = projects_df.drop_duplicates(subset=["project_id"])

In [1698]:
projects_df = projects_df.rename(columns={"baseSeries": "base_series", "комплекс": "name", "project_id": "id"})

In [1699]:
cursor.execute(open("sql/xx_projects.sql", "r").read())
conn.commit()

In [1700]:
execute_values(conn, projects_df, "projects")

execute_values() done


In [1701]:
set_sequence(conn, "projects")

(132,)


<h1>Узлы</h1>

In [1702]:
df10 = pd.DataFrame(df9["объект"].str.split('.', 1).tolist(),
                                 columns = ['f_p','o_p'])

In [1703]:
df10 = df10.fillna("")

In [1704]:
df11 = df9.join(df10)

In [1705]:
df11["node_feature"] = df11["baseSeries"] + df11["nodeCode"]

In [1706]:
enc = OrdinalEncoder()
res = enc.fit_transform(np.array(df11["node_feature"]).reshape(-1, 1))

In [1707]:
s = pd.DataFrame(pd.Series(res.flatten()) + 1)
s = s.rename(columns={0: "node_id"})

In [1708]:
df12 = df11.join(s)

In [1709]:
nodes_df = df12[["node_id", "f_p", "project_id", "nodeCode"]]

In [1710]:
chiefEngineerIds = [92, 123, 144, 169, 201, 270, 305, 311, 368, 370, 396, 400, 443, 461, 462, 604, 608]

In [1711]:
nodes_df["chief_engineer"] = random.choices(chiefEngineerIds, k=len(nodes_df))

C:\Users\sigla\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [1712]:
nodes_df = nodes_df.rename(columns={"node_id": "id", "f_p": "name", "nodeCode": "code", "chief_engineer": "chief_engineer_id"})

In [1713]:
nodes_df = nodes_df.drop_duplicates(subset=["id"])

In [1714]:
cursor.execute(open("sql/xx_nodes.sql", "r").read())
conn.commit()

In [1715]:
execute_values(conn, nodes_df, "nodes")

execute_values() done


In [1716]:
set_sequence(conn, "nodes")

(208,)


<h1>Подузлы</h1>

In [1717]:
df13 = pd.DataFrame(df12["o_p"].str.split('.', 1).tolist(),
                                 columns = ['s_p','m_p'])

In [1718]:
df13 = df13.fillna("")

In [1719]:
df13["s_p"] = df13["s_p"].str.strip()
df13["m_p"] = df13["m_p"].str.strip()

In [1720]:
df14 = df12.join(df13)

In [1721]:
df14["subnode_feature"] = df11["node_feature"] + df11["subnodeCode"]

In [1722]:
enc = OrdinalEncoder()
res = enc.fit_transform(np.array(df14["subnode_feature"]).reshape(-1, 1))

In [1723]:
s = pd.DataFrame(pd.Series(res.flatten()) + 1)
s = s.rename(columns={0: "subnode_id"})

In [1724]:
df15 = df14.join(s)

In [1725]:
subnodes_df = df15[["subnode_id", "node_id", "subnodeCode", "s_p"]]

In [1726]:
subnodes_df = subnodes_df.rename(columns={"subnode_id": "id", "s_p": "name", "subnodeCode": "code"})

In [1727]:
subnodes_df = subnodes_df.drop_duplicates(subset=["id"])

In [1728]:
cursor.execute(open("sql/xx_subnodes.sql", "r").read())
conn.commit()

In [1729]:
execute_values(conn, subnodes_df, "subnodes")

execute_values() done


In [1730]:
set_sequence(conn, "subnodes")

(241,)


<h1>Марки</h1>

In [1731]:
df15 = df15.drop(["обозн", "комплекс", "объект", "level_0", "ОЗ_марки"], axis=1)

In [1732]:
df15["t_эксплуат"] = df15["t_эксплуат"].str.replace("минус ", "-")
df15["t_эксплуат"] = df15["t_эксплуат"].str.replace("плюс ", "")
df15["t_эксплуат"] = df15["t_эксплуат"].str.replace("+", "")
df15["t_эксплуат"] = df15["t_эксплуат"].str.replace("- 39", "-39")
df15 = df15[df15["t_эксплуат"] != "-"]
df15 = df15[df15["t_эксплуат"] != "минус"]
df15["t_эксплуат"] = df15["t_эксплуат"].astype("float64")

In [1733]:
df15["об_марки"] = df15["об_марки"].str.strip()
df15["об_марки"] = df15["об_марки"].str.replace("KM", "КМ")
df15 = df15[df15["об_марки"] != "по"]

In [1734]:
df15 = df15.drop(["об_марки"], axis=1)

In [1735]:
df15 = df15.drop(["project_id", "f_p", "o_p", "s_p", "subnode_feature", "node_feature", "subnodeCode"], axis=1)

In [1736]:
df15 = df15.drop(["baseSeries", "nodeCode", "node_id"], axis=1)

In [1737]:
df15["прим"] = df15["прим"].str.strip()
df15["прим"] = df15["прим"].str.replace("\r\n       ", " ")

In [1738]:
df15 = df15.drop(["ДопКод"], axis=1)

In [1739]:
df15["категория_пр"] = 3

In [1740]:
df15["агрессивность"] += 1
df15["группа_газов"] += 1
df15["материал"] += 1
df15["вп_болты"] += 1

# Part
df15.loc[df15["тип_ЛКМ"] == "ПФ", "тип_ЛКМ"] = 1
df15.loc[df15["тип_ЛКМ"] == "АУ", "тип_ЛКМ"] = 2
df15.loc[df15["тип_ЛКМ"] == "ЭЭ", "тип_ЛКМ"] = 3
df15.loc[df15["тип_ЛКМ"] == "МА", "тип_ЛКМ"] = 4
df15.loc[df15["тип_ЛКМ"] == "ПВ", "тип_ЛКМ"] = 11

In [1741]:
df15 = df15.drop_duplicates(subset=["code", "subnode_id"])

In [1742]:
df15_operating_cond = df15.copy()

In [1743]:
# df15 = df15.drop(["ВесПоВМП", "ВыпЗдСм", "10ХСНД", "ТекстЗдСм", "ДопОбъемы", "коэф_надежн", "агрессивность", "t_эксплуат", "зона_эксплуат", "группа_газов", "материал", "вп_болты"], axis=1)

In [1744]:
# df16 = df15.rename(columns={"код_отд": "department_id",
#                             "подп1": "signed1_id",
#                             "подп2": "signed2_id",
#                             "гл_спец": "chief_specialist_id",
#                             "рук_гр": "group_leader_id",
#                             "н_контр": "main_builder_id",
#                             "дата_выд": "issue_date",
#                             "кол_томов": "num_of_volumes",
#                             "дата_ред": "edited_date",
#                             "тип_ЛКМ": "paintwork_type",
#                             "прим": "note",
#                             "категория_пр": "fire_hazard_category_id",
#                             "п_транспорт": "p_transport",
#                             "п_площадки": "p_site",
#                             "m_p": "name",
#                        })

In [1745]:
df15 = df15.rename(columns={"код_отд": "department_id",
                            "подп1": "signed1_id",
                            "подп2": "signed2_id",
                            "гл_спец": "chief_specialist_id",
                            "рук_гр": "group_leader_id",
                            "н_контр": "main_builder_id",
                            "дата_выд": "issue_date",
                            "кол_томов": "num_of_volumes",
                            "дата_ред": "edited_date",
                            "коэф_надежн": "safety_coeff",
                            "t_эксплуат": "operating_temp",
                            "зона_эксплуат": "operating_zone",
                            "группа_газов": "gas_group",
                            "агрессивность": "aggressiveness",
                            "материал": "material",
                            "тип_ЛКМ": "paintwork_type",
                            "прим": "note",
                            "категория_пр": "fire_hazard_category_id",
                            "вп_болты": "high_tensile_bolts",
                            "п_транспорт": "p_transport",
                            "п_площадки": "p_site",
                            "10ХСНД": "xcnd",
                            "ТекстЗдСм": "task_text",
                            "ДопОбъемы": "additional_text",
                            "ВесПоВМП": "vmp_weight",
                            "ВыпЗдСм": "impl_3d_estimate",
                            "m_p": "name"
                       })

In [1746]:
df15["issue_date"] = df15["issue_date"].astype(object).where(df15["issue_date"].notnull(), None)
df15["edited_date"] = df15["edited_date"].astype(object).where(df15["edited_date"].notnull(), None)
df15 = df15.where(pd.notnull(df15), None)

In [1747]:
df15["chief_specialist_id"] = df15["chief_specialist_id"].replace(0, None)
df15["group_leader_id"] = df15["group_leader_id"].replace(0, None)
df15["main_builder_id"] = df15["main_builder_id"].replace(0, None)

In [1748]:
df16 = df15.drop(["vmp_weight", "impl_3d_estimate", "xcnd", "task_text", "additional_text", "safety_coeff", "aggressiveness", "operating_temp", "operating_zone", "gas_group", "material", "high_tensile_bolts"], axis=1)

In [1749]:
df16 = df16.reset_index(drop=True)

In [1750]:
df16["id"].to_csv("mark_ids.csv")

In [1751]:
cursor.execute(open("sql/7.sql", "r").read())
conn.commit()

In [1752]:
execute_values(conn, df16, "marks")

execute_values() done


In [1753]:
set_sequence(conn, "marks")

(5596,)


<h1>Условия эксплуатации марки</h1>

In [1754]:
df_op_cond = df15_operating_cond.loc[:, ["id", "коэф_надежн", "агрессивность", "t_эксплуат", "зона_эксплуат", "группа_газов", "материал", "тип_ЛКМ", "вп_болты"]]

In [1755]:
df_op_cond["t_эксплуат"] = df_op_cond["t_эксплуат"].fillna(-34)

In [1756]:
df_op_cond = df_op_cond.rename(columns={"id": "mark_id",
                            "коэф_надежн": "safety_coeff",
                            "t_эксплуат": "temperature",
                            "зона_эксплуат": "operating_area_id",
                            "группа_газов": "gas_group_id",
                            "агрессивность": "env_aggressiveness_id",
                            "материал": "construction_material_id",
                            "тип_ЛКМ": "paintwork_type_id",
                            "вп_болты": "high_tensile_bolts_type_id"
                       })

In [1757]:
cursor.execute(open("sql/y_mark_operating_conditions.sql", "r").read())
conn.commit()

In [1758]:
execute_values(conn, df_op_cond, "mark_operating_conditions")

execute_values() done


<h1>Задание на смету</h1>

In [1759]:
df_task = df15.loc[:, ["id", "task_text", "additional_text"]]

In [1760]:
df_task["task_text"] = df_task["task_text"].str.strip()

In [1761]:
df['обозн'] = 'Разработать сметную документацию к чертежам ' + df['обозн'].astype(str) + '\r\nСостав и объемы работ:'

In [1762]:
df_task.loc[df_task["task_text"].isna(), "task_text"] = list(df.loc[df["код_марки"].isin(df_task.loc[df_task["task_text"].isna(), "id"]), "обозн"])

In [1763]:
df_task = df_task.rename(columns={"id": "mark_id"})

In [1764]:
df_task["task_text"] = df_task["task_text"].str.replace("\r\n", "\n")

In [1765]:
df_task["additional_text"] = df_task["additional_text"].str.replace("\r\n", "\n")

In [1766]:
cursor.execute(open("sql/y_estimate_task.sql", "r").read())
conn.commit()

In [1767]:
execute_values(conn, df_task, "estimate_task")

execute_values() done


<h1>Выпуски спецификаций</h1>

In [1768]:
df = pd.read_sql_query('SELECT * FROM Pro_t_londonSQL.dbo.ВыпСпец', db_conn)

In [1769]:
df = df.replace("", np.nan)

In [1770]:
df = df.drop_duplicates(subset=["код_марки", "выпуск"])

In [1771]:
df = df.drop(["вкл_вып"], axis=1)

In [1772]:
df["текущий"] = df["текущий"].map(lambda x: True if (x == 1) else False)

In [1773]:
df.insert(0, 'id', range(1, len(df) + 1))

In [1774]:
df = df.rename(columns={"код_марки": "mark_id",
                        "выпуск": "num",
                        "дата_созд": "created_date",
                        "текущий": "is_current",
                        "прим": "note"
                       })

In [1775]:
df["created_date"] = df["created_date"].astype(object).where(df["created_date"].notnull(), datetime(2000, 1, 1))
df = df.where(pd.notnull(df), None)

In [1776]:
m_ids = pd.read_csv("mark_ids.csv")
m_ids = m_ids.drop(["Unnamed: 0"], axis=1)
m_ids = m_ids.values.flatten().tolist()

In [1777]:
df = df[df["mark_id"].isin(m_ids)]

In [1778]:
df.loc[:, ["id", "mark_id", "num"]].to_csv("specs.csv")

In [1779]:
cursor.execute(open("sql/3.sql", "r").read())
conn.commit()

In [1780]:
execute_values(conn, df, "specifications")

execute_values() done


In [1781]:
set_sequence(conn, "specifications")

(3848,)


<h1>Листы</h1>

In [1782]:
df = pd.read_sql_query('SELECT * FROM Pro_t_londonSQL.dbo.Листы', db_conn)

In [1783]:
df = df.replace("", np.nan)

In [1784]:
df = df[df["номер"].isna() == False]

In [1785]:
df["тип_док"] = df["тип_док"] + 1

In [1786]:
df["вып"] = df["вып"].map(lambda x: np.nan if x == 0 else x)

In [1787]:
df["пров"] = df["пров"].map(lambda x: np.nan if x == 0 else x)

In [1788]:
df["норм"] = df["норм"].map(lambda x: np.nan if x == 0 else x)

In [1789]:
df = df.drop(["шифр_вып", "шифр_пров", "шифр_норм"], axis=1)

In [1790]:
df["формат"] = df["формат"].map(lambda x: 1.0 if np.isnan(x) else x)

In [1791]:
df = df.drop_duplicates(subset=["код_марки", "номер", "тип_док"])

In [1792]:
df = df[df["вып"].isna() == False]

In [1793]:
df["название"] = df["название"].str.strip()

In [1794]:
for i in range(0, len(df)):
    try:
        arr = df["название"][i].split()
        new_str = ' '.join(arr)
        df.at[i, "название"] = new_str
    except Exception:
        pass

In [1795]:
df.insert(0, 'id', range(1, len(df) + 1))

In [1796]:
df = df.rename(columns={"код_марки": "mark_id",
                        "номер": "num",
                        "название": "name",
                        "формат": "form",
                        "вып": "creator_id",
                        "пров": "inspector_id",
                        "норм": "norm_contr_id",
                        "тип_док": "type_id",
                        "выпуск": "release_num",
                        "листов": "num_of_pages",
                        "прим": "note"
                       })

In [1797]:
df = df.where(pd.notnull(df), None)

In [1798]:
m_ids = pd.read_csv("mark_ids.csv")
m_ids = m_ids.drop(["Unnamed: 0"], axis=1)
m_ids = m_ids.values.flatten().tolist()

In [1799]:
e_ids = pd.read_csv("employee_ids.csv")
e_ids = e_ids.drop(["Unnamed: 0"], axis=1)
e_ids = e_ids.values.flatten().tolist()

In [1800]:
df = df[df["mark_id"].isin(m_ids)]
df.loc[df["creator_id"].isin(e_ids) == False, "creator_id"] = None
df.loc[df["inspector_id"].isin(e_ids) == False, "inspector_id"] = None
df.loc[df["norm_contr_id"].isin(e_ids) == False, "norm_contr_id"] = None

In [1801]:
df["release_num"] = df["release_num"].fillna(0)
df["num_of_pages"] = df["num_of_pages"].fillna(0)

In [1802]:
df = df[df["creator_id"].isna() == False]
df = df[df["name"].isna() == False]

In [1803]:
cursor.execute(open("sql/6.sql", "r").read())
conn.commit()

In [1804]:
execute_values(conn, df, "docs")

execute_values() done


In [1805]:
set_sequence(conn, "docs")

(27022,)


<h1>Обозначения марок</h1>

In [1806]:
df = pd.read_sql_query('SELECT * FROM Pro_t_londonSQL.dbo.Об_марок', db_conn)

In [1807]:
df = df.iloc[1:, :]
df = df.iloc[:-1, :]
df.loc[len(df) + 1] = ["КМД", "Конструкции металлические деталировочные"]

In [1808]:
df.insert(0, 'id', range(1, len(df) + 1))

In [1809]:
df = df.rename(columns={"об_марки": "designation",
                        "наим_марки": "name"
                       })

In [1810]:
cursor.execute(open("sql/9.sql", "r").read())
conn.commit()

In [1811]:
execute_values(conn, df, "mark_designations")

execute_values() done


In [1812]:
set_sequence(conn, "mark_designations")

(3,)


<h1>Прилагаемые документы марки</h1>

In [1813]:
df = pd.read_sql_query('SELECT * FROM Pro_t_londonSQL.dbo.Од_прил', db_conn)

In [1814]:
df = df.replace("", np.nan)

In [1815]:
df = df.drop_duplicates(subset=["код_марки", "обозн_прил"])

In [1816]:
df = df.where(pd.notnull(df), None)

In [1817]:
df.insert(0, 'id', range(1, len(df) + 1))

In [1818]:
df = df.rename(columns={"код_марки": "mark_id",
                        "наим_прил": "name",
                        "прим": "note",
                        "обозн_прил": "designation"
                       })

In [1819]:
m_ids = pd.read_csv("mark_ids.csv")
m_ids = m_ids.drop(["Unnamed: 0"], axis=1)
m_ids = m_ids.values.flatten().tolist()
df = df[df["mark_id"].isin(m_ids)]

In [1820]:
cursor.execute(open("sql/12.sql", "r").read())
conn.commit()

In [1821]:
execute_values(conn, df, "attached_docs")

execute_values() done


In [1822]:
set_sequence(conn, "attached_docs")

(3965,)


<h1>Согласования</h1>

In [1823]:
df = pd.read_sql_query('SELECT * FROM Pro_t_londonSQL.dbo.Од_согл', db_conn)

In [1824]:
df = df.replace("", np.nan)

In [1825]:
df = df.drop(["отдел1", "отдел2", "отдел3", "отдел4", "отдел5", "отдел6", "отдел7"], axis=1)

In [1826]:
p1 = df.iloc[:, [0,1]]
p1 = p1.rename(columns={"исп1": "исп"})

p2 = df.iloc[:, [0,2]]
p2 = p2.rename(columns={"исп2": "исп"})

p3 = df.iloc[:, [0,3]]
p3 = p3.rename(columns={"исп3": "исп"})

p4 = df.iloc[:, [0,4]]
p4 = p4.rename(columns={"исп4": "исп"})

p5 = df.iloc[:, [0,5]]
p5 = p5.rename(columns={"исп5": "исп"})

p6 = df.iloc[:, [0,6]]
p6 = p6.rename(columns={"исп6": "исп"})

p7 = df.iloc[:, [0,7]]
p7 = p7.rename(columns={"исп7": "исп"})

In [1827]:
df = pd.concat([p1, p2, p3, p4, p5, p6, p7], ignore_index=True)

In [1828]:
df = df.dropna()
df = df[df["исп"] != 0]

In [1829]:
df = df.drop_duplicates()

In [1830]:
df.insert(0, 'id', range(1, len(df) + 1))

In [1831]:
df = df.rename(columns={"код_марки": "mark_id",
                        "исп": "employee_id"
                       })

In [1832]:
m_ids = pd.read_csv("mark_ids.csv")
e_ids = pd.read_csv("employee_ids.csv")

In [1833]:
m_ids = m_ids.drop(["Unnamed: 0"], axis=1)
e_ids = e_ids.drop(["Unnamed: 0"], axis=1)

In [1834]:
m_ids = m_ids.values.flatten().tolist()
e_ids = e_ids.values.flatten().tolist()

In [1835]:
df = df[df["mark_id"].isin(m_ids)]
df = df[df["employee_id"].isin(e_ids)]

In [1836]:
cursor.execute(open("sql/13.sql", "r").read())
conn.commit()

In [1837]:
execute_values(conn, df, "mark_approvals")

execute_values() done


In [1838]:
set_sequence(conn, "mark_approvals")

(3987,)


<h1>Ссылочные документы марки (id sequence is out of sync)</h1>

In [1839]:
df = pd.read_sql_query('SELECT * FROM Pro_t_londonSQL.dbo.Од_ссыл', db_conn)

In [1840]:
df = df.replace("", np.nan)

In [1841]:
df= df[df["шифр_сд"] != "1"]
df["шифр_сд"] = df["шифр_сд"].str.upper()

In [1842]:
m_ids = pd.read_csv("mark_ids.csv")
m_ids = m_ids.drop(["Unnamed: 0"], axis=1)
m_ids = m_ids.values.flatten().tolist()
df = df[df["код_марки"].isin(m_ids)]

In [1843]:
for i in range(1, 301):
    df = df.append(pd.Series(), ignore_index=True)

C:\Users\sigla\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


In [1844]:
df.loc[:, "код_марки"] = random.choices(m_ids, k=300)
df.loc[:, "шифр_сд"] = random.choices(range(1, 300), k=300)

In [1845]:
df = df.drop_duplicates(subset=["код_марки", "шифр_сд"])

In [1846]:
df.insert(0, 'id', range(1, len(df) + 1))

In [1847]:
df = df.rename(columns={"код_марки": "mark_id",
                        "шифр_сд": "linked_doc_id",
                        "прим": "note"
                       })

In [1848]:
cursor.execute(open("sql/14.sql", "r").read())
conn.commit()

In [1849]:
execute_values(conn, df, "mark_linked_docs")

execute_values() done


In [1850]:
set_sequence(conn, "mark_linked_docs")

(300,)


<h1>Учет дополнительных работ</h1>

In [1851]:
df = pd.read_sql_query('SELECT * FROM Pro_t_londonSQL.dbo.Од_вып_доп', db_conn)

In [1852]:
df = df.replace("", np.nan)

In [1853]:
df = df.drop(["шифр_исп"], axis=1)

In [1854]:
df = df[df["код_исп"].isna() == False]

In [1855]:
df = df.drop_duplicates(subset=["код_марки", "код_исп"])

In [1856]:
m_ids = pd.read_csv("mark_ids.csv")
m_ids = m_ids.drop(["Unnamed: 0"], axis=1)
m_ids = m_ids.values.flatten().tolist()
df = df[df["код_марки"].isin(m_ids)]

In [1857]:
df.insert(0, 'id', range(1, len(df) + 1))

In [1858]:
df = df.rename(columns={"код_марки": "mark_id",
                        "код_исп": "employee_id",
                        "расчет": "valuation",
                        "заказ": "metal_order",
                       })

In [1859]:
cursor.execute(open("sql/11.sql", "r").read())
conn.commit()

In [1860]:
execute_values(conn, df, "additional_work")

execute_values() done


In [1861]:
set_sequence(conn, "additional_work")

(750,)


<h1>Типовые конструкции</h1>

In [1862]:
df = pd.read_sql_query('SELECT * FROM Pro_t_londonSQL.dbo.Тс_типовые', db_conn)

In [1863]:
df = df.drop(["upsize_ts"], axis=1)

In [1864]:
df = df[df["имя_тк"].isna() == False]

In [1865]:
df["имя_тк"] = df["имя_тк"].str.strip()

<h2>Типовые наименования типовых конструкций</h2>

In [1866]:
st_name_df = pd.DataFrame(data=df["имя_тк"].unique(), columns=["name"])

In [1867]:
st_name_df.loc[3, "name"] = "Рельсы крановые КР-70 сталь К63"
st_name_df = st_name_df[st_name_df["name"] != "КР-70"]
st_name_df.loc[5, "name"] = "Рельсы крановые Р-43"
st_name_df = st_name_df[st_name_df["name"] != "Оцинкованная сталь ГОСТ14918-80*"]
st_name_df.loc[11, "name"] = "Рельсы крановые КР-70"
st_name_df.loc[12, "name"] = "Детали крепления крановых рельсов"
st_name_df = st_name_df[st_name_df["name"] != "Крановый рельс КР-70"]
st_name_df.loc[16, "name"] = "Накладка 5 на рельс Р-24 по ГОСТ 8141-56"

In [1868]:
st_name_df.insert(0, 'id', range(1, len(st_name_df) + 1))

In [1869]:
cursor.execute(open("sql/yyy_1.sql", "r").read())
conn.commit()

In [1870]:
execute_values(conn, st_name_df, "standard_construction_names")

execute_values() done


--------------------

In [1871]:
m_ids = pd.read_csv("mark_ids.csv")
m_ids = m_ids.drop(["Unnamed: 0"], axis=1)
m_ids = m_ids.values.flatten().tolist()
df = df[df["код_марки"].isin(m_ids)]

In [1872]:
specs = pd.read_csv("specs.csv")
specs = specs.drop(["Unnamed: 0"], axis=1)

In [1873]:
for index, row in df.iterrows():
    try:
        df.loc[index, "specification_id"] = specs.loc[(specs["mark_id"] == row["код_марки"]) & (specs["num"] == row["выпуск"]), "id"].values[0]
    except:
        pass

In [1874]:
df = df.drop(["код_марки", "выпуск"], axis=1)

In [1875]:
df.loc[24, "кол_тк"] = 1

In [1876]:
# https://wikiweight.info/strmat/massa-ankerov-hilti.html
df.loc[28, "вес_тк"] = 0.056
df.loc[32, "вес_тк"] = 0.056

In [1877]:
df.loc[25, "вес_тк"] = 0.024
df.loc[29, "вес_тк"] = 0.024
df.loc[25, "кол_тк"] = 360
df.loc[29, "кол_тк"] = 360
df = df[df["сч_тк"] != 54]
df = df[df["сч_тк"] != 58]

In [1878]:
df = df.rename(columns={"сч_тк": "id",
                        "имя_тк": "name",
                        "кол_тк": "num",
                        "лист_тк": "sheet",
                        "вес_тк": "weight"
                       })

In [1879]:
df = df.where(pd.notnull(df), None)

In [1880]:
cursor.execute(open("sql/47.sql", "r").read())

In [1881]:
execute_values(conn, df, "standard_constructions")

execute_values() done


In [1882]:
set_sequence(conn, "standard_constructions")

(60,)


<h1>Виды конструкций марки</h1>

In [1883]:
df = pd.read_sql_query('SELECT * FROM Pro_t_londonSQL.dbo.Тс_Виды_к', db_conn)

In [1884]:
df = df.drop_duplicates()

In [1885]:
df = df.drop(["upsize_ts"], axis=1)

In [1886]:
df = df.dropna(subset=["кт_окраски", "кол_тип_констр", "имя_к", "Шифр_к", "Шифр_пк"])

In [1887]:
m_ids = pd.read_csv("mark_ids.csv")
m_ids = m_ids.drop(["Unnamed: 0"], axis=1)
m_ids = m_ids.values.flatten().tolist()
df = df[df["код_марки"].isin(m_ids)]

In [1888]:
specs = pd.read_csv("specs.csv")
specs = specs.drop(["Unnamed: 0"], axis=1)

In [1889]:
df.insert(1, "specification_id", 0, True) 

In [1890]:
for index, row in df.iterrows():
    try:
        df.loc[index, "specification_id"] = specs.loc[(specs["mark_id"] == row["код_марки"]) & (specs["num"] == row["выпуск"]), "id"].values[0]
    except:
        pass

In [1891]:
df = df[df["specification_id"] != 0]

In [1892]:
df = df.drop(["код_марки", "выпуск"], axis=1)

In [1893]:
df["Шифр_пк"] = df["Шифр_пк"].replace(0, np.nan)

In [1894]:
df = df.drop(["вкл", "окраска", "огнезащита", "ПределОС", "ПредСост_несущ", "ПредСост_целост", "ПредСост_тепло_изол", "ПредСост_тепло_поток", "ПредСост_газ", "статус_к", "ТеорРасходОЗ"], axis=1)

In [1895]:
df = df.rename(columns={"сч_констр": "id",
                        "имя_к": "name",
                        "Шифр_к": "type_id",
                        "Шифр_пк": "subtype_id",
                        "расценка": "valuation",
                        "шифр_тип_альбома": "standard_album_code",
                        "кол_тип_констр": "num_of_standard_constructions",
                        "притупление": "has_edge_blunting",
                        "динамика": "has_dynamic_load",
                        "фланцы": "has_flanged_connections",
                        "контроль_сварки": "welding_control_id",
                        "кт_окраски": "paintwork_coeff",
                       })

In [1896]:
df = df.where(pd.notnull(df), None)

In [1897]:
df = df[df["type_id"] < 88]

In [1898]:
df = df[df["type_id"] > 0]

In [1899]:
df = df.drop_duplicates(subset=["specification_id", "name", "paintwork_coeff"])

In [1900]:
cursor.execute(open("sql/46.sql", "r").read())
conn.commit()

In [1901]:
execute_values(conn, df, "constructions")

execute_values() done


In [1902]:
set_sequence(conn, "constructions")

(22555,)


<h1>Справочник болтов, диаметры</h1>

In [1903]:
df = pd.read_sql_query('SELECT * FROM Pro_t_londonSQL.dbo.Спр_болтов', db_conn)

In [1904]:
df = df.drop(["upsize_ts"], axis=1)

In [1905]:
df = df.rename(columns={"код_д": "id",
                        "диаметр": "diameter",
                        "масса_г": "nut_weight",
                        "сталь_ш": "washer_steel",
                        "масса_ш": "washer_weight",
                        "толщина_ш": "washer_thickness",
                        "ТУ_б": "bolt_tech_spec",
                        "кл_пр_б": "strength_class",
                        "ТУ_г": "nut_tech_spec",
                        "ТУ_ш": "washer_tech_spec",
                       })

In [1906]:
cursor.execute(open("sql/20.sql", "r").read())
conn.commit()

In [1907]:
execute_values(conn, df, "bolt_diameters")

execute_values() done


In [1908]:
set_sequence(conn, "bolt_diameters")

(6,)


<h1>Справочник болтов, длины</h1>

In [1909]:
df = pd.read_sql_query('SELECT * FROM Pro_t_londonSQL.dbo.Спр_болтов_дл', db_conn)

In [1910]:
df.insert(0, 'id', range(1, len(df) + 1))

In [1911]:
df = df.rename(columns={"код_д": "diameter_id",
                        "длина_б": "length",
                        "длина_нар": "screw_length",
                        "масса_б": "weight"
                       })

In [1912]:
cursor.execute(open("sql/21.sql", "r").read())
conn.commit()

In [1913]:
execute_values(conn, df, "bolt_lengths")

execute_values() done


In [1914]:
set_sequence(conn, "bolt_lengths")

(75,)


<h1>Высокопрочные болты видов конструкций</h1>

In [1915]:
df = pd.read_sql_query('SELECT * FROM Pro_t_londonSQL.dbo.Тс_болты', db_conn)

In [1916]:
c_ids = pd.read_csv("construction_ids.csv")
c_ids = c_ids.drop(["Unnamed: 0"], axis=1)
c_ids = c_ids.values.flatten().tolist()

In [1917]:
df = df[df["сч_констр"].isin(c_ids)]

In [1918]:
df.loc[:, "код_диаметра"] = df.loc[:, "код_диаметра"].replace(0, np.nan)

In [1919]:
df = df.dropna()

In [1920]:
df = df.rename(columns={"сч_болта": "id",
                        "сч_констр": "construction_id",
                        "код_диаметра": "diameter_id",
                        "пакет": "packet",
                        "кол": "num",
                        "гаек": "nut_num",
                        "шайб": "washer_num",
                       })

In [1921]:
cursor.execute(open("sql/45.sql", "r").read())
conn.commit()

In [1922]:
execute_values(conn, df, "construction_bolts")

execute_values() done


In [1923]:
set_sequence(conn, "construction_bolts")

(43,)


<h1>Виды профилей</h1>

In [1924]:
df = pd.read_sql_query('SELECT * FROM Pro_t_londonSQL.dbo.Спр_видов_п', db_conn)

In [1925]:
df["Имя_в"] = df["Имя_в"].str.strip()
df["Прим"] = df["Прим"].str.strip()

In [1926]:
df = df.drop_duplicates(subset=["Имя_в"])

In [1927]:
df = df.rename(columns={"Шифр_в": "id",
                        "Имя_в": "name",
                        "Прим": "note"
                       })

In [1928]:
df = df.sort_values(by=["id"])

In [1929]:
df = df.reset_index(drop=True)

In [1930]:
df.loc[:, ["id"]].to_csv("prof.csv")

In [1931]:
cursor.execute(open("sql/23.sql", "r").read())
conn.commit()

In [1932]:
execute_values(conn, df, "profile_classes")

execute_values() done


In [1933]:
set_sequence(conn, "profile_classes")

(123,)


<h1>Типы профилей</h1>

In [1934]:
df = pd.read_sql_query('SELECT * FROM Pro_t_londonSQL.dbo.Спр_типов_проф', db_conn)

In [1935]:
df["Имя_т"] = df["Имя_т"].str.strip()

In [1936]:
df = df.rename(columns={"Тип": "id",
                        "Имя_т": "name"
                       })

In [1937]:
cursor.execute(open("sql/42.sql", "r").read())
conn.commit()

In [1938]:
execute_values(conn, df, "profile_types")

execute_values() done


In [1939]:
set_sequence(conn, "profile_types")

(11,)


<h1>Профили</h1>

In [1940]:
df = pd.read_sql_query('SELECT * FROM Pro_t_londonSQL.dbo.Спр_проф_полн', db_conn)

In [1941]:
df = df.replace("", np.nan)

In [1942]:
df = df[df["Вес"].isna() == False]

In [1943]:
df["Имя_п"] = df["Имя_п"].str.strip()
df["Имя_п0"] = df["Имя_п0"].str.strip()

In [1944]:
df = df.drop(["Имя_п0"], axis=1)

In [1945]:
df = df[df["Разрешен"]]
df = df.drop(["Разрешен"], axis=1)

In [1946]:
df = df.drop(["дл_имя", "Дл1", "Дл2", "Дл3", "Дл4", "Дл5"], axis=1)

In [1947]:
df = df.drop(["Об1", "Об2", "Об3", "Об4", "Об5"], axis=1)

In [1948]:
df = df[(df["Имя_п"] != "200*160*7") | (df["Символ"].isna() == False)]

In [1949]:
df = df.drop_duplicates(subset=["Вид_п", "Имя_п", "Символ"])

In [1950]:
df.loc[df["Вид_п"] == 16, "Символ"] = "-"
df.loc[df["Вид_п"] == 17, "Символ"] = "-"

In [1951]:
df_2 = pd.read_sql_query('SELECT * FROM Pro_t_londonSQL.dbo.Тс_элементы', db_conn)

In [1952]:
df_2["Имя_п"] = df_2["Имя_п"].str.strip()
df_2["Символ"] = df_2["Символ"].str.strip()

In [1953]:
c_ids = pd.read_csv("construction_ids.csv")
c_ids = c_ids.drop(["Unnamed: 0"], axis=1)
c_ids = c_ids.values.flatten().tolist()

In [1954]:
df_2 = df_2[df_2["Сч_констр"].isin(c_ids)]

In [1955]:
p_ids = pd.read_csv("prof.csv")
p_ids = p_ids.drop(["Unnamed: 0"], axis=1)
p_ids = p_ids.values.flatten().tolist()

In [1956]:
df_2 = df_2[df_2["Вид_п"].isin(p_ids)]

In [1957]:
df_2 = df_2[df_2["Символ"].isna() == False]

In [1958]:
df_2 = df_2.drop(["МаркаОЗ", "АрифмВыр"], axis=1)

In [1959]:
df_2 = df_2.drop(["ПерекрПерим", "КолВСеч", "Поз", "ГруппаПТ"], axis=1)

In [1960]:
for index, row in df_2.iterrows():
    try:
        df_2.loc[index, "profile_id"] = df.loc[(df["Вид_п"] == row["Вид_п"]) & (df["Имя_п"] == row["Имя_п"]) & (df["Символ"] == row["Символ"]), "Профиль"].values[0]
    except Exception as e:
        pass

In [1961]:
df_null = df_2.loc[df_2['profile_id'].isna()]

In [1962]:
df_null = df_null.drop(["Сч_констр", "Сч_поз", "Сталь", "Длина", "статус", "profile_id"], axis=1)

In [1963]:
df_null = df_null.rename(columns={"Вид_п": "class_id",
                        "Имя_п": "name",
                        "Символ": "symbol",
                        "Вес": "weight",
                        "Поверхн": "area",
                        "Тип_п": "type_id"
                       })

In [1964]:
df_null.insert(0, 'id', range(1962, len(df_null) + 1962))

In [1965]:
df = df.where(pd.notnull(df), None)

In [1966]:
df = df.rename(columns={"Профиль": "id",
                        "Вид_п": "class_id",
                        "Имя_п": "name",
                        "Символ": "symbol",
                        "Вес": "weight",
                        "Поверхн": "area",
                        "Тип_п": "type_id"
                       })

In [1967]:
df = df.append(df_null, ignore_index = True)

In [1968]:
df[df.duplicated(subset=["class_id", "name", "symbol"], keep="first")]
df = df.drop_duplicates(subset=["class_id", "name", "symbol"])

In [1969]:
df = df[df["id"] != 3194]

In [1970]:
df.loc[df["id"] == 2755, "weight"] = 4.396
df.loc[df["id"] == 2755, "area"] = 0.00288

In [1971]:
df.loc[:, ["id", "class_id", "name", "symbol"]].to_csv("profiles.csv")

In [1972]:
cursor.execute(open("sql/33.sql", "r").read())
conn.commit()

In [1973]:
execute_values(conn, df, "profiles")

execute_values() done


In [1974]:
set_sequence(conn, "profiles")

(3306,)


<h1>Марки стали</h1>

In [1975]:
df = pd.read_sql_query('SELECT * FROM Pro_t_londonSQL.dbo.Спр_стали', db_conn)

In [1976]:
df = df.replace("", np.nan)

In [1977]:
df = df.drop([71])

In [1978]:
df["Имя_с"] = df["Имя_с"].str.strip()
df["Стандарт"] = df["Стандарт"].str.strip()

In [1979]:
df = df.rename(columns={"Шифр_с": "id",
                        "Имя_с": "name",
                        "Стандарт": "standard",
                        "Прочность": "strength"
                       })

In [1980]:
df = df.where(pd.notnull(df), None)

In [1981]:
cursor.execute(open("sql/41.sql", "r").read())
conn.commit()

In [1982]:
execute_values(conn, df, "steel")

execute_values() done


In [1983]:
set_sequence(conn, "steel")

(73,)


<h1>Перечень элементов</h1>

In [1984]:
df = pd.read_sql_query('SELECT * FROM Pro_t_londonSQL.dbo.Тс_элементы', db_conn)

In [1985]:
df["Имя_п"] = df["Имя_п"].str.strip()
df["Символ"] = df["Символ"].str.strip()

In [1986]:
c_ids = pd.read_csv("construction_ids.csv")
c_ids = c_ids.drop(["Unnamed: 0"], axis=1)
c_ids = c_ids.values.flatten().tolist()

In [1987]:
df = df[df["Сч_констр"].isin(c_ids)]

In [1988]:
p_ids = pd.read_csv("prof.csv")
p_ids = p_ids.drop(["Unnamed: 0"], axis=1)
p_ids = p_ids.values.flatten().tolist()

In [1989]:
df = df[df["Вид_п"].isin(p_ids)]

In [1990]:
df.loc[df["Сталь"] == 72, "Сталь"] = 36

In [1991]:
df = df[df["Символ"].isna() == False]

In [1992]:
df = df.drop(["МаркаОЗ", "АрифмВыр"], axis=1)

In [1993]:
df = df.drop(["ПерекрПерим", "КолВСеч", "Поз", "ГруппаПТ"], axis=1)

In [1994]:
profiles = pd.read_csv("profiles.csv")
profiles = profiles.drop(["Unnamed: 0"], axis=1)

In [1995]:
df.loc[df["Сч_поз"] == 179045, "Имя_п"] = "140*4"
df.loc[df["Сч_поз"] == 179034, "Имя_п"] = "140*4"

In [1996]:
for index, row in df.iterrows():
    try:
        df.loc[index, "profile_id"] = profiles.loc[(profiles["class_id"] == row["Вид_п"]) & (profiles["name"] == row["Имя_п"]) & (profiles["symbol"] == row["Символ"]), "id"].values[0]
    except:
        pass

In [1997]:
df = df.drop(["Имя_п", "Символ", "Вес", "Поверхн", "Тип_п", "статус", "Вид_п"], axis=1)

In [1998]:
df = df.rename(columns={"Сч_констр": "construction_id",
                        "Сч_поз": "id",
                        "Сталь": "steel_id",
                        "Длина": "length"
                       })

In [1999]:
cursor.execute(open("sql/48.sql", "r").read())
conn.commit()

In [2000]:
execute_values(conn, df, "construction_elements")

execute_values() done


In [2001]:
set_sequence(conn, "construction_elements")

(187834,)


<h1>Разделы общих указаний</h1>

In [2002]:
df = pd.DataFrame({'id': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17],
                   'name': ["Разработано на основании",
                            "Разработано в соответствии",
                            "Перечень технических регламентов",
                            "Высотная отметка",
                            "Климатические условия",
                            "Конструктивные решения",
                            "Характеристики конструкций",
                            "Материал конструкций",
                            "Указания по разработке КМД и ППР",
                            "Указания по изготовлению и монтажу",
                            "Крепление элементов",
                            "Соединение элементов",
                            "Антикоррозионная защита",
                            "Огнезащита конструкций",
                            "Перечень скрытых видов работ",
                            "Перечень ответственных конструкций",
                            "Коэффициент трудозатрат"],
                   'order_num': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]})

In [2003]:
cursor.execute(open("sql/37_2.sql", "r").read())
conn.commit()

In [2004]:
execute_values(conn, df, "general_data_sections")

execute_values() done


In [2005]:
set_sequence(conn, "general_data_sections")

(17,)


<h1>Пункты общих указаний марки</h1>

In [2006]:
df = pd.DataFrame({
#     'id': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73],
                   'mark_id': [4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320, 4320],
                   'section_id': [1, 1, 1, 1, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 8, 8, 9, 9, 10, 10, 10, 10, 11, 11, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 13, 13, 13, 13, 13, 13, 13, 13, 13, 15, 15, 15, 15],
                   'text': ["Рабочая документация разработана на основании:",
                            "- поручения от 28.02.2020 № АГЛ-35/0362",
                            "- письма от 05.03.2020 № УПиН-35/0431",
                            "- рекомендаций, выполненного специалистами \"АО МАГНИТОГОРСКИЙ ГИПРОМЕЗ\"",
                            "Рабочая документация разработана в соответствии с техническими условиями, требованиями действующих технических регламентов, стандартов, сводов правил и других документов, содержащих установленные требования.",
                            "Перечень технических регламентов и нормативных документов, в соответствии с требованиями которых разработана рабочая документация",
                            "- Федеральный закон от 30.12.2009 № 384-ФЗ \"Технический регламент о безопасности зданий и сооружений\"",
                            "- Федеральный закон от 22.07.2008 № 123-ФЗ \"Технический регламент о требованиях пожарной безопасности\"",
                            "- СП 16.13330.2017 \"Стальные конструкции. Актуализированная редакция СНиП II-23-81*\"",
                            "- СП 20.13330.2016 \"Нагрузки и воздействия. Актуализированная редакция СНиП 2.01.07-85*\"",
                            "- СП 28.13330.2017 \"Защита строительных конструкций от коррозии. Актуализированная редакция СНиП 2.03.11-85\"",
                            "- СП 53-102-2004 \"Общие правила проектирования стальных конструкций\"",
                            "- СП 70.13330.2012 \"Несущие и ограждающие конструкции. Актуализированная редакция СНиП 3.03.01-87\"",
                            "- ГОСТ 23118-2012 \"Конструкции стальные строительные. Общие технические условия\"",
                            "- ГОСТ 27751-2014 \"Надежность строительных конструкций и оснований. Основные положения\"",
                            "- ГОСТ 9.402-2004 \"Единая система защиты от коррозии и старения (ЕСЗКС). Покрытия лакокрасочные. Подготовка металлических поверхностей к окрашиванию\"",
                            "Высотные отметки абсолютные.",
                            "Климатические условия",
                            "- климатический район – IB",
                            "- расчетная температура наружного воздуха наиболее холодных суток обеспеченностью 0,98 – минус 43 °C",
                            "- нормативное значение веса снегового покрова на 1 м^2 горизонтальной поверхности земли для г. Магнитогорска (согласно СП 20.13330-2016, приложение K, табл. K1) – 1,3 кПа (130 кг/м^2)",
                            "- нормативное значение ветрового давления для II района (согласно СП 20.13330-2016, п. 11.14, табл. 11.1) – 0,30 кПа (30 кг/м^2)",
                            "- зона влажности – сухая",
                            "- расчетная сейсмическая активность – 5 баллов",
                            "Конструктивные решения",
                            "# Чертежи марки \"КМ\" являются исходным материалом для разработки деталировочных чертежей марки \"КМД\"",
                            "# В данной рабочей документации разработано восстановление пролетных строений (усиление ферм пролетных строений, металлоконструкций по верхним и нижним поясам ферм) и восстановление антикоррозионного покрытия существующих элементов пролетных строений галереи конвейера № 706-1 УУК и КДИ аглоцеха ГОП ПАО \"ММК\"",
                            "# Все работы должны выполняться специализированной организацией, имеющей свидетельство СРО на данный вид деятельности и имеющие допуск по электробезопасности",
                            "# Работы производить по специально разработанному и согласованному в установленном порядке со всеми инспектирующими организациями и эксплуатацией цеха проекту производства работ (ППР) с соблюдением требований действующих нормативных документов и обязательным учетом фактического состояния существующих конструкций",
                            "# При выполнении работ принять меры по обеспечению целостности и сохранности существующих сооружений и коммуникаций, находящихся в зоне производства работ",
                            "# При производстве сварочных работ предусмотреть комплекс организационно-технических мероприятий по обеспечению пожарной безопасности",
                            "# На всех этапах ведения демонтажных и монтажных работ обеспечить устойчивость и неизменяемость положения существующих конструкций",
                            "# Перед началом работ необходимо выполнить очистку конструкций в зоне ремонта от мусора, грязи, технологической пыли",
                            "# Исполнитель несет ответственность за соответствие выполненных работ чертежам. Всякое отступление от чертежей при производстве работ должно быть согласовано с организацией-разработчиком данной рабочей документации",
                            "Характеристики конструкций",
                            "- коэффициент надежности по ответственности – {}",
                            "- степень агрессивного воздействия среды – {}",
                            "- расчетная температура эксплуатации – {}",
                            "Материал конструкций",
                            "Марки сталей элементов конструкций приняты в зависимости от группы конструкций с учетом расчетной температуры и приведены в таблицах \"Ведомость элементов\", на схемах расположения элементов конструкций, на чертежах элементов и узлов конструкций и в спецификации металлопроката",
                            "Указания по разработке КМД и ППР",
                            "Отступления от рабочих чертежей КМ не допускаются. В случае необходимости эти отступления должны быть согласованы с организацией-разработчиком рабочих чертежей марки \"КМ\"",
                            "Указания по изготовлению и монтажу металлоконструкций",
                            "# Изготовление металлоконструкций производить в соответствии с требованиями ГОСТ 23118-2012, СП 53-101-98 \"Изготовление и контроль качества стальных строительных конструкций\", указаниями в чертежах марки \"КМ\"",
                            "# Монтаж металлоконструкций производить в соответствии с требованиями СП 70.13330.2012 \"Несущие и ограждающие конструкции. Актуализированная редакция СНиП 3.03.01-87\", СНИП 12-04-2002 \"Безопасность труда в строительстве. Часть 2. Строительное производство\" и указаниями в чертежах марки \"КМ\"",
                            "# Отклонения металлоконструкций при изготовлении и монтаже не должны превышать предельных значений, указанных в СП 53-101-98, СП 70.13330.2012 и в чертежах марки \"КМ\" данного основного комплекта",
                            "Крепление элементов",
                            "Минимальное усилие для расчетного прикрепления элементов – 5 тс",
                            "Соединение элементов металлоконструкций",
                            "- сварные",
                            "- на болтах нормальной точности по ГОСТ Р ИСО 4014-2013",
                            "# Материалы для сварки принимать по таблице Г.1 СП 16.13330.2017 в зависимости от марок стали свариваемых элементов",
                            "# Монтажные швы выполнять ручной сваркой ГОСТ 9467-75 \"Электроды покрытые металлические для ручной дуговой сварки конструкционных и теплоустойчивых сталей. Типы\":\nДля сталей C245 сварку производить электродами типа 342A",
                            "# Применение прерывистых швов не допускается",
                            "# Контроль качества монтажных сварных соединений должен осуществляться в соответствии с требованиями п. 10.4 СП 70.13330.2012",
                            "# Болты класса точности B (нормальной точности) M20 приняты по ГОСТ Р ИСО 4014-2013 \"Болты с шестигранной головкой. Классы точности A и B\", класса прочности 8.8 по ГОСТ ISO 898-1-2014 \"Механические свойства крепежных изделий из углеродистых и легированных сталей. Часть 1. Болты, винты и шпильки установленных классов прочности с крупным и мелким шагом резьбы\"",
                            "# Гайки тип 1 класса точности B приняты по ГОСТ ISO 4032-2014. При работе болтов на срез и растяжение класс прочности гаек принят в соответствии с классом прочности болтов: 5 – при 5.6",
                            "Шайбы приняты круглые по ГОСТ 11371-78 исполнения 1 класса точности A",
                            "Для болтов класса точности B диаметр отверстий в конструкциях, работающих на срез и смятие, принят d=dв+2",
                            "В расчетных соединениях для болтов класса точности B разность номинальных диаметров отверстий и болтов не должна превышать 3 мм с учетом предельных отклонений диаметра отверстия и их овальности 0; +0,6; и 0; +1,0 по табл. 2 ГОСТ 23118-2012",
                            "Антикоррозионная защита",
                            "# Антикоррозионная защита запроектирована в соответствии с требованиями СП 28.13330.2017 \"Защита строительных конструкций от коррозии\"",
                            "# Антикоррозионную защиту производить в соответствии с требованиями СП 72.13330.2016 \"Защита строительных конструкций и сооружений от коррозии. СНиП 3.04.03-85\" и ГОСТ 12.3.005-75 \"Система стандартов безопасности труда (ССБТ). Работы окрасочные. Общие требования безопасности\"",
                            "# Подготовка поверхностей перед окраской должна осуществляться в соответствии с требованиями ГОСТ 9.402-2004 путем удаления ржавчины и прокатной окалины с помощью ручного или механизированного абразивного инструмента и очистки жировых и прочих загрязнений с помощью волосяных кистей или ветоши, смоченных в уайт-спирите. Перед нанесением антикоррозионного покрытия поверхности конструкций обеспылить",
                            "# Защита металлоконструкций от коррозии осуществляется окраской лакокрасочными материалами группы III: эмалью XB-7141 по ТУ 2313-016-75351875-2005 в 4 слоя по 2 слоям грунтовки XC-010 по ТУ 6-21-7-89",
                            "Общая толщина покрытия – 160 мкм",
                            "Степень очистки поверхности стальных конструкций от окислов перед окраской – 2 по ГОСТ 9.402-2004",
                            "Допускается замена лакокрасочных материалов на сертифицированные материалы с аналогичными техническими характеристиками",
                            "Цвет окраски антикоррозионного покрытия принимать в соответствии с цветом окраски существующих конструкций",
                            "Перечень видов работ, на которые должны быть составлены акты освидетельствования скрытых работ:",
                            "- подготовка поверхности металлоконструкций под окраску",
                            "- приемка грунтовочного слоя антикоррозионного покрытия",
                            "- визуальный контроль качества сварных швов"],
                   'order_num': [1, 2, 3, 4, 1, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 1, 1, 2, 3, 4, 5, 6, 7, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 1, 2, 3, 4, 1, 2, 1, 2, 1, 2, 3, 4, 1, 2, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 1, 2, 3, 4, 5, 6, 7, 8, 9, 1, 2, 3, 4]})


In [2007]:
df.insert(0, 'id', range(1, len(df) + 1))

In [2008]:
cursor.execute(open("sql/35_xx.sql", "r").read())
conn.commit()

In [2009]:
execute_values(conn, df, "mark_general_data_points")

execute_values() done


In [2010]:
set_sequence(conn, "mark_general_data_points")

(73,)


<h1>Первый запуск до этой клетки, потом нужно программно добавить пользователей</h1>

<h1>Пункты общих указаний пользователя (id seq fail)</h1>

In [2011]:
df = pd.DataFrame({
#     'id': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73],
                   'user_id': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
                   'section_id': [1, 1, 1, 1, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 8, 8, 9, 9, 10, 10, 10, 10, 11, 11, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 13, 13, 13, 13, 13, 13, 13, 13, 13, 15, 15, 15, 15],
                   'text': ["Рабочая документация разработана на основании:",
                            "- поручения от 28.02.2020 № АГЛ-35/0362",
                            "- письма от 05.03.2020 № УПиН-35/0431",
                            "- рекомендаций, выполненного специалистами \"АО МАГНИТОГОРСКИЙ ГИПРОМЕЗ\"",
                            "Рабочая документация разработана в соответствии с техническими условиями, требованиями действующих технических регламентов, стандартов, сводов правил и других документов, содержащих установленные требования.",
                            "Перечень технических регламентов и нормативных документов, в соответствии с требованиями которых разработана рабочая документация",
                            "- Федеральный закон от 30.12.2009 № 384-ФЗ \"Технический регламент о безопасности зданий и сооружений\"",
                            "- Федеральный закон от 22.07.2008 № 123-ФЗ \"Технический регламент о требованиях пожарной безопасности\"",
                            "- СП 16.13330.2017 \"Стальные конструкции. Актуализированная редакция СНиП II-23-81*\"",
                            "- СП 20.13330.2016 \"Нагрузки и воздействия. Актуализированная редакция СНиП 2.01.07-85*\"",
                            "- СП 28.13330.2017 \"Защита строительных конструкций от коррозии. Актуализированная редакция СНиП 2.03.11-85\"",
                            "- СП 53-102-2004 \"Общие правила проектирования стальных конструкций\"",
                            "- СП 70.13330.2012 \"Несущие и ограждающие конструкции. Актуализированная редакция СНиП 3.03.01-87\"",
                            "- ГОСТ 23118-2012 \"Конструкции стальные строительные. Общие технические условия\"",
                            "- ГОСТ 27751-2014 \"Надежность строительных конструкций и оснований. Основные положения\"",
                            "- ГОСТ 9.402-2004 \"Единая система защиты от коррозии и старения (ЕСЗКС). Покрытия лакокрасочные. Подготовка металлических поверхностей к окрашиванию\"",
                            "Высотные отметки абсолютные.",
                            "Климатические условия",
                            "- климатический район – IB",
                            "- расчетная температура наружного воздуха наиболее холодных суток обеспеченностью 0,98 – минус 43 °C",
                            "- нормативное значение веса снегового покрова на 1 м^2 горизонтальной поверхности земли для г. Магнитогорска (согласно СП 20.13330-2016, приложение K, табл. K1) – 1,3 кПа (130 кг/м^2)",
                            "- нормативное значение ветрового давления для II района (согласно СП 20.13330-2016, п. 11.14, табл. 11.1) – 0,30 кПа (30 кг/м^2)",
                            "- зона влажности – сухая",
                            "- расчетная сейсмическая активность – 5 баллов",
                            "Конструктивные решения",
                            "# Чертежи марки \"КМ\" являются исходным материалом для разработки деталировочных чертежей марки \"КМД\"",
                            "# В данной рабочей документации разработано восстановление пролетных строений (усиление ферм пролетных строений, металлоконструкций по верхним и нижним поясам ферм) и восстановление антикоррозионного покрытия существующих элементов пролетных строений галереи конвейера № 706-1 УУК и КДИ аглоцеха ГОП ПАО \"ММК\"",
                            "# Все работы должны выполняться специализированной организацией, имеющей свидетельство СРО на данный вид деятельности и имеющие допуск по электробезопасности",
                            "# Работы производить по специально разработанному и согласованному в установленном порядке со всеми инспектирующими организациями и эксплуатацией цеха проекту производства работ (ППР) с соблюдением требований действующих нормативных документов и обязательным учетом фактического состояния существующих конструкций",
                            "# При выполнении работ принять меры по обеспечению целостности и сохранности существующих сооружений и коммуникаций, находящихся в зоне производства работ",
                            "# При производстве сварочных работ предусмотреть комплекс организационно-технических мероприятий по обеспечению пожарной безопасности",
                            "# На всех этапах ведения демонтажных и монтажных работ обеспечить устойчивость и неизменяемость положения существующих конструкций",
                            "# Перед началом работ необходимо выполнить очистку конструкций в зоне ремонта от мусора, грязи, технологической пыли",
                            "# Исполнитель несет ответственность за соответствие выполненных работ чертежам. Всякое отступление от чертежей при производстве работ должно быть согласовано с организацией-разработчиком данной рабочей документации",
                            "Характеристики конструкций",
                            "- коэффициент надежности по ответственности – {}",
                            "- степень агрессивного воздействия среды – {}",
                            "- расчетная температура эксплуатации – {}",
                            "Материал конструкций",
                            "Марки сталей элементов конструкций приняты в зависимости от группы конструкций с учетом расчетной температуры и приведены в таблицах \"Ведомость элементов\", на схемах расположения элементов конструкций, на чертежах элементов и узлов конструкций и в спецификации металлопроката",
                            "Указания по разработке КМД и ППР",
                            "Отступления от рабочих чертежей КМ не допускаются. В случае необходимости эти отступления должны быть согласованы с организацией-разработчиком рабочих чертежей марки \"КМ\"",
                            "Указания по изготовлению и монтажу металлоконструкций",
                            "# Изготовление металлоконструкций производить в соответствии с требованиями ГОСТ 23118-2012, СП 53-101-98 \"Изготовление и контроль качества стальных строительных конструкций\", указаниями в чертежах марки \"КМ\"",
                            "# Монтаж металлоконструкций производить в соответствии с требованиями СП 70.13330.2012 \"Несущие и ограждающие конструкции. Актуализированная редакция СНиП 3.03.01-87\", СНИП 12-04-2002 \"Безопасность труда в строительстве. Часть 2. Строительное производство\" и указаниями в чертежах марки \"КМ\"",
                            "# Отклонения металлоконструкций при изготовлении и монтаже не должны превышать предельных значений, указанных в СП 53-101-98, СП 70.13330.2012 и в чертежах марки \"КМ\" данного основного комплекта",
                            "Крепление элементов",
                            "Минимальное усилие для расчетного прикрепления элементов – 5 тс",
                            "Соединение элементов металлоконструкций",
                            "- сварные",
                            "- на болтах нормальной точности по ГОСТ Р ИСО 4014-2013",
                            "# Материалы для сварки принимать по таблице Г.1 СП 16.13330.2017 в зависимости от марок стали свариваемых элементов",
                            "# Монтажные швы выполнять ручной сваркой ГОСТ 9467-75 \"Электроды покрытые металлические для ручной дуговой сварки конструкционных и теплоустойчивых сталей. Типы\":\nДля сталей C245 сварку производить электродами типа 342A",
                            "# Применение прерывистых швов не допускается",
                            "# Контроль качества монтажных сварных соединений должен осуществляться в соответствии с требованиями п. 10.4 СП 70.13330.2012",
                            "# Болты класса точности B (нормальной точности) M20 приняты по ГОСТ Р ИСО 4014-2013 \"Болты с шестигранной головкой. Классы точности A и B\", класса прочности 8.8 по ГОСТ ISO 898-1-2014 \"Механические свойства крепежных изделий из углеродистых и легированных сталей. Часть 1. Болты, винты и шпильки установленных классов прочности с крупным и мелким шагом резьбы\"",
                            "# Гайки тип 1 класса точности B приняты по ГОСТ ISO 4032-2014. При работе болтов на срез и растяжение класс прочности гаек принят в соответствии с классом прочности болтов: 5 – при 5.6",
                            "Шайбы приняты круглые по ГОСТ 11371-78 исполнения 1 класса точности A",
                            "Для болтов класса точности B диаметр отверстий в конструкциях, работающих на срез и смятие, принят d=dв+2",
                            "В расчетных соединениях для болтов класса точности B разность номинальных диаметров отверстий и болтов не должна превышать 3 мм с учетом предельных отклонений диаметра отверстия и их овальности 0; +0,6; и 0; +1,0 по табл. 2 ГОСТ 23118-2012",
                            
                            
                            "Антикоррозионная защита",
                            "# Антикоррозионная защита запроектирована в соответствии с требованиями СП 28.13330.2017 \"Защита строительных конструкций от коррозии\"",
                            "# Антикоррозионную защиту производить в соответствии с требованиями СП 72.13330.2016 \"Защита строительных конструкций и сооружений от коррозии. СНиП 3.04.03-85\" и ГОСТ 12.3.005-75 \"Система стандартов безопасности труда (ССБТ). Работы окрасочные. Общие требования безопасности\"",
                            "# Подготовка поверхностей перед окраской должна осуществляться в соответствии с требованиями ГОСТ 9.402-2004 путем удаления ржавчины и прокатной окалины с помощью ручного или механизированного абразивного инструмента и очистки жировых и прочих загрязнений с помощью волосяных кистей или ветоши, смоченных в уайт-спирите. Перед нанесением антикоррозионного покрытия поверхности конструкций обеспылить",
                            "# Защита металлоконструкций от коррозии осуществляется окраской лакокрасочными материалами группы III: эмалью XB-7141 по ТУ 2313-016-75351875-2005 в 4 слоя по 2 слоям грунтовки XC-010 по ТУ 6-21-7-89",
                            "Общая толщина покрытия – 160 мкм",
                            "Степень очистки поверхности стальных конструкций от окислов перед окраской – 2 по ГОСТ 9.402-2004",
                            "Допускается замена лакокрасочных материалов на сертифицированные материалы с аналогичными техническими характеристиками",
                            "Цвет окраски антикоррозионного покрытия принимать в соответствии с цветом окраски существующих конструкций",
                            "Перечень видов работ, на которые должны быть составлены акты освидетельствования скрытых работ:",
                            "- подготовка поверхности металлоконструкций под окраску",
                            "- приемка грунтовочного слоя антикоррозионного покрытия",
                            "- визуальный контроль качества сварных швов"],
                   'order_num': [1, 2, 3, 4, 1, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 1, 1, 2, 3, 4, 5, 6, 7, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 1, 2, 3, 4, 1, 2, 1, 2, 1, 2, 3, 4, 1, 2, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 1, 2, 3, 4, 5, 6, 7, 8, 9, 1, 2, 3, 4]})

In [2012]:
df.insert(0, 'id', range(1, len(df) + 1))

In [2013]:
cursor.execute(open("sql/35_2.sql", "r").read())
conn.commit()

Will fail first time. Need to create users.

In [2014]:
execute_values(conn, df, "general_data_points")

execute_values() done


In [2015]:
set_sequence(conn, "general_data_points")

(73,)


In [2016]:
conn.close()